# Testing of hists/dataframe.py

In [1]:
%load_ext autoreload
%autoreload 2
from functools import partial
import pandas as pd
import numpy as np
import uproot
import awkward as ak
import sys
sys.path.append("..")
from hists.dataframe import *
from hists.shower_axis import *

In [2]:
#input_file = '/grid_mnt/vol_home/llr/cms/cuisset/hgcal/testbeam18/clue3d-dev/src/ClusteringAnalysis/CLUE_clusters_single.root'
input_file = '/data_cms_upgrade/cuisset/testbeam18/clue3d/v30/cmssw/data/CLUE_clusters.root'
#input_file = '/data_cms_upgrade/cuisset/testbeam18/clue3d/v20/cmssw/sim_proton_v7/CLUE_clusters.root'

In [3]:
for array_i in uproot.iterate(input_file + ":clusters", step_size="100MB", library="ak"):
    series = array_i
    break

In [4]:
comp = DataframeComputations(series)

In [5]:
pd.set_option('display.min_rows', 20)
pd.set_option('display.max_rows', 500)

In [20]:
index=weights = pd.Series(dict(zip(range(1, 29), [2, 3] + list(range(26))))).rename_axis("rechits_layer").rename("weights")
weights

rechits_layer
1      2
2      3
3      0
4      1
5      2
6      3
7      4
8      5
9      6
10     7
11     8
12     9
13    10
14    11
15    12
16    13
17    14
18    15
19    16
20    17
21    18
22    19
23    20
24    21
25    22
26    23
27    24
28    25
Name: weights, dtype: int64

In [21]:
comp.rechits_totalReconstructedEnergyPerEventLayer_allLayers()#.join(weights, on="rechits_layer")

rechits_energy_sum_perLayer  beamEnergy   
eventInternal rechits_layer                                            
0             1                                 4.312928       300.0  \
              2                                 5.497473       300.0   
              3                                10.904448       300.0   
              4                                12.052125       300.0   
              5                                16.710476       300.0   
              6                                14.199676       300.0   
              7                                13.610519       300.0   
              8                                12.423681       300.0   
              9                                13.673567       300.0   
              10                                9.741046       300.0   
...                                                  ...         ...   
9860          19                                0.137070        20.0   
              20                                0.096109        20.0   
              21                                0.082707        20.0   
              22                                0.088554        20.0   
              23                                0.089437        20.0   
              24                                0.000000        20.0   
              25                                0.000000        20.0   
              26                                0.000000        20.0   
              27                                0.000000        20.0   
              28                                0.000000        20.0   

                             synchrotronBeamEnergy   
eventInternal rechits_layer                          
0             1                             287.18  \
              2                             287.18   
              3                             287.18   
              4                             287.18   
              5                             287.18   
              6                             287.18   
              7                             287.18   
              8                             287.18   
              9                             287.18   
              10                            287.18   
...                                            ...   
9860          19                             20.00   
              20                             20.00   
              21                             20.00   
              22                             20.00   
              23                             20.00   
              24                             20.00   
              25                             20.00   
              26                             20.00   
              27                             20.00   
              28                             20.00   

                             rechits_energy_sum_perLayer_fractionOfSynchrotronBeamEnergy  
eventInternal rechits_layer                                                               
0             1                                                       0.015018            
              2                                                       0.019143            
              3                                                       0.037971            
              4                                                       0.041967            
              5                                                       0.058188            
              6                                                       0.049445            
              7                                                       0.047394            
              8                                                       0.043261            
              9                                                       0.047613            
              10                                                      0.033920            
...                                  

In [24]:
df

rechits_id      rechits_energy          
cumcount                             0    1              0         1
eventInternal rechits_layer                                         
0             1                      9   17       0.806782  0.511084
              2                     40   39       1.546924  1.218609
              3                     82   81       1.970687  1.665135
              4                    128  127       3.080546  1.291326
              5                    179  178       3.259795  1.856267
              6                    247  246       4.070849  2.318207
              7                    311  310       3.541324  1.225369
              8                    373  372       3.316550  1.005916
              9                    438  437       2.639365  1.117544
              10                   508  507       2.775838  0.633792
...                                ...  ...            ...       ...
9860          14                   192  189       0.061596  0.041449
              15                   200  207       0.060696  0.048389
              16                   213  211       0.113021  0.029140
              17                   219  215       0.026821  0.020282
              18                   222  223       0.055850  0.019200
              19                   224  225       0.044907  0.032442
              20                   229  230       0.048872  0.034499
              21                   234  232       0.042154  0.029246
              22                   235   -1       0.088554 -1.000000
              23                   236   -1       0.089437 -1.000000

[255175 rows x 4 columns]

In [31]:
df.rechits_energy[0]/df.rechits_energy[1]

eventInternal  rechits_layer
0              1                1.578570
               2                1.269419
               3                1.183500
               4                2.385568
               5                1.756103
               6                1.756034
               7                2.890006
               8                3.297045
               9                2.361754
               10               4.379732
                                  ...   
9860           14               1.486052
               15               1.254342
               16               3.878539
               17               1.322394
               18               2.908784
               19               1.384216
               20               1.416602
               21               1.441327
               22              -0.088554
               23              -0.089437
Length: 255175, dtype: float32

In [27]:
df[(df.rechits_id[1] >= 0) &(df.rechits_energy[0] > 20*df.rechits_energy[1])].rechits_id[0]

eventInternal  rechits_layer
7              2                  8
               6                 86
9              4                 33
19             2                  6
28             6                 63
43             2                  5
               6                 82
48             1                  1
50             4                 36
68             2                 10
                               ... 
9671           2                  3
9688           2                  6
9699           20               223
9702           2                  3
               4                 18
9713           2                  4
9749           2                  1
9751           1                  1
9753           2                  4
9773           2                  6
Name: 0, Length: 1385, dtype: int64

In [30]:
comp.rechits.query("eventInternal==9773 and rechits_layer == 2")

beamEnergy  rechits_x  rechits_y  rechits_z   
eventInternal rechits_id                                                
9773          5                 20.0   3.626411   0.662958    14.7675  \
              6                 20.0   3.626411  -0.462042    14.7675   
              7                 20.0   2.728303   0.077848    14.7675   
              8                 20.0   1.753815  -0.484652    14.7675   
              9                 20.0   0.779450  -2.172518    14.7675   
              10                20.0  -0.195159  -1.609896    14.7675   
              11                20.0   5.651765  -1.609896    14.7675   
              12                20.0   4.677278   0.077848    14.7675   
              13                20.0   4.677278  -1.047396    14.7675   

                          rechits_energy  rechits_layer  rechits_rho   
eventInternal rechits_id                                               
9773          5                 0.031619              2     0.714590  \
              6                 1.312811              2     1.363103   
              7                 0.040449              2     0.717124   
              8                 0.008920              2     0.029145   
              9                 0.018631              2     0.035118   
              10                0.032974              2     0.042290   
              11                0.024762              2     0.032678   
              12                0.012682              2     0.692814   
              13                0.015833              2     0.690960   

                          rechits_delta  rechits_nearestHigher   
eventInternal rechits_id                                         
9773          5            1.071892e+00                      7  \
              6            3.402823e+38                     -1   
              7            1.047893e+00                      6   
              8            1.125181e+00                      7   
              9            1.125347e+00                     10   
              10           3.402823e+38                     -1   
              11           1.125181e+00                     13   
              12           1.181440e+00                      6   
              13           1.125244e+00                     12   

                          rechits_pointType  
eventInternal rechits_id                     
9773          5                           0  
              6                           1  
              7                           0  
              8                           0  
              9                           0  
              10                          1  
              11                          0  
              12                          0  
              13                          0

In [36]:
comp.rechits_ratioFirstToSecondMostEnergeticHitsPerLayer

eventInternal  rechits_layer
0              1                1.578570
               2                1.269419
               3                1.183500
               4                2.385568
               5                1.756103
               6                1.756034
               7                2.890006
               8                3.297045
               9                2.361754
               10               4.379732
                                  ...   
9860           12               2.078147
               13               1.455728
               14               1.486052
               15               1.254342
               16               3.878539
               17               1.322394
               18               2.908784
               19               1.384216
               20               1.416602
               21               1.441327
Length: 239171, dtype: float32

In [35]:
(comp.rechits_ratioFirstToSecondMostEnergeticHitsPerLayer > 5).sum()

32659